In [1]:
from rdflib.plugins.stores import sparqlstore
import rdflib
from rdflib import URIRef, Literal, Graph, Dataset

import serialization
import discourse

import html
#import loader
import discourse
import pandas as pd
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, RDFS
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot
from IPython.core.display import HTML

import uuid

from itertools import chain

def visualize(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
    png = dg.create_png()

    display(Image(png))

def t2rdflibg(triples): #triples to rdflib graph
    g = Graph()
    for t in triples:
        g.add(t)
    return g


def nan2None(value):
    if pd.isnull(value) :
        return None
    else:
        return value


def get_data_rows(filename):
    data_rows_df = pd.read_csv(filename)
    data_rows=[]
    for i,r in data_rows_df.iterrows():
        r_d = {k:nan2None(v) for k,v in dict(r).items()}

        data_rows.append(r_d)
    del data_rows_df
    return data_rows

serial_key = { "Business Domain" : "ModelDomain", 
               "Model" : "Model", 
               "ModelType" : "ModelType", 
               "Entity" : "Class", 
               "Attribute" : "Attribute", 
               "DataType" : "DataType", 
               "Context" : "Context", 
               "SubContext" : "SubContext", 
               "Relationship" : "Relationship", 
               "Entity From" : "FromClass",
               "Attribute From" : "FromAttribute", 
               "Cardinality From" : "FromCardinality",
               "Entity To" : "ToClass",
               "Attribute To" : "ToAttribute",
               "Cardinality To" : "ToCardinality"
}

jena = sparqlstore.SPARQLUpdateStore("http://localhost:3030/models/query",context_aware=True)
jena.open(("http://localhost:3030/models/query", "http://localhost:3030/models/update"))

# Create a Dataset
ds = Dataset(store=jena, default_union=True, default_graph_base="http://base.raw")
# Define a named graph within the dataset, this will contain all nodes contained within the graph referenced, or none if it doesn't exist yet.
sg = ds.graph(URIRef("http://config"))

master_g = ds.graph(URIRef("http://master"))
discourse_g = ds.graph(URIRef("http://discourse"))



* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * WARNING: ObjectProperty http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MappingMetaTarget belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, owl.topObjectProperty]; I'm trying to fix it...
* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit:/usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/rd/q3bhk6_n56x75m05pp2sq7dr0000gn/T/tmpbzuh114v


Serialization.Meta [Serialization.SerializationMetaData]
Serialization.Mapping [Serialization.SerializationSpecs]
Serialization.BatchDefinition [Serialization.SerializationSpecs]
Serialization.SerializationSpecs [Serialization.SerializationMetaData]
Serialization.BatchNode [Serialization.DataLoad]
Serialization.DataLoad [Serialization.SerializationMetaData]
Serialization.SerializationMetaData [owl.Thing]
Serialization.DataPropertyMapping [Serialization.Mapping]
Serialization.MetaDataProperty [Serialization.Meta]
Serialization.EntityMapping [Serialization.Mapping]
Serialization.MetaClass [Serialization.Meta]
Serialization.JobNode [Serialization.DataLoad]
Serialization.MetaProperty [Serialization.Meta]
Serialization.PropertyMapping [Serialization.Mapping]
Serialization.RowNode [Serialization.DataLoad]
Serialization.Serialization [Serialization.SerializationSpecs]


* Owlready2 * HermiT took 0.5102579593658447 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [2]:
model_data = pd.read_csv(r"mdm_v11_3_pdm_additional.csv").fillna("")
model_data.columns=['Sequence', 'ModelDomain', 'Model', 'ModelType', 'Class',
       'ClassDescription', 'Attribute', 'AttributeDescription', 'DataType',
       'Context', 'SubContext', 'Nulls', 'Is PK']

In [3]:
def key_filter(keys,xlist=None):
    s_keys = sorted(keys)
    if xlist is not None:
        if not tuple([k for k in s_keys if k not in xlist]) == tuple():
            s_keys = tuple([k for k in s_keys if k not in xlist])
        else:
            s_keys = tuple(s_keys)

    else:
        s_keys = tuple([k for k in s_keys])
    return s_keys

from functools import partial
from collections import Counter
fkf = partial(key_filter, xlist=["LANG_TP_CD"])

In [4]:
raw_keys = dict(model_data[model_data['Is PK']=="Yes"].groupby("Class")['Attribute'].agg(fkf))
[(k,v) for k,v in Counter(raw_keys.values()).items() if len(k)>1]

[(('BKTHASH', 'RECNO', 'SRCRECNO'), 1),
 (('CMPSEQNO', 'RECNO'), 1),
 (('SRCID', 'SRCRECNO'), 1),
 (('CITY_NAME',
   'GIVEN_NAME_ONE',
   'LAST_NAME',
   'P_CITY_NAME',
   'P_GIVEN_NAME_ONE',
   'P_LAST_NAME'),
  1)]

In [5]:
# Contents here block the inversion of the table/pk mapping dict
print([(k,v) for k,v in Counter(raw_keys.values()).items() if v>1])
# FOR IBM products, sharing the same PK implies parent/sub-class relationships (though there's)
# no systematic method for determining which one is the parent.
sorted([(k,v) for k,v in raw_keys.items() if v in [k for k,v in Counter(raw_keys.values()).items() if v>1]], key=lambda x : x[1])
manual_key_masters = { ('CONT_ID',) : 'CONTACT', 
                       ('HOLDING_ID',): 'HOLDING', 
                      ('LOCATION_GROUP_ID',) : 'LOCATIONGROUP', 
                      ('PAYMENT_SOURCE_ID', ): 'PAYMENTSOURCE',
                      ('PPREF_ID',) : 'PRIVPREF',
                      ('PRODUCT_ID',) : 'PRODUCT', 
                      ('SUSPECT_ID',) : 'SUSPECT' , 
                      ('LANG_TP_CD', ) : 'CDLANGTP'
                     }

key_map = {**{v:k for k,v in raw_keys.items()}, **manual_key_masters}


[(('LOCATION_GROUP_ID',), 3), (('PAYMENT_SOURCE_ID',), 4), (('CONT_ID',), 6), (('PRODUCT_ID',), 5), (('HOLDING_ID',), 3), (('PPREF_ID',), 3), (('SUSPECT_ID',), 2)]


In [6]:
def att_link(row):
    attribute=row['Attribute']
    table = row['Class']
    t_att = tuple([attribute])
    t_val = key_map.get(t_att)
    relationship = str(table) + " --> " + str(t_val)
    if t_val is not None and t_val != table:
        return pd.Series((relationship, table, attribute, t_val, t_att[0]))
    else:
        return pd.Series((None, None, None, None, None))
    

In [7]:
model_data[['Relationship','FromClass','FromAttribute','ToClass','ToAttribute']] = model_data.apply(att_link, axis=1)
model_data

,Sequence,ModelDomain,Model,ModelType,Class,ClassDescription,Attribute,AttributeDescription,DataType,Context,SubContext,Nulls,Is PK,Relationship,FromClass,FromAttribute,ToClass,ToAttribute
0,0,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,ADDACTIONTYPE,The ADDACTIONTYPE table identifies an action t...,ADD_ACTION_ID,"A unique, system-generated key that identifies...",BIGINT,,,Not Null,Yes,None,None,None,None,None
1,1,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,ADDACTIONTYPE,The ADDACTIONTYPE table identifies an action t...,MATCH_RELEV_TP_CD,Identifies the CONTACT match relevancies - sco...,BIGINT,,,Not Null,No,ADDACTIONTYPE --> CDMATCHRELEVTP,ADDACTIONTYPE,MATCH_RELEV_TP_CD,CDMATCHRELEVTP,MATCH_RELEV_TP_CD
2,2,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,ADDACTIONTYPE,The ADDACTIONTYPE table identifies an action t...,SUSP_REASON_TP_CD,Describes the critical data that was considere...,BIGINT,,,Not Null,No,ADDACTIONTYPE --> CDSUSPECTREASONTP,ADDACTIONTYPE,SUSP_REASON_TP_CD,CDSUSPECTREASONTP,SUSP_REASON_TP_CD
3,3,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,ADDACTIONTYPE,The ADDACTIONTYPE table identifies an action t...,ORG_TP_CD,Identifies the classification of the organizat...,BIGINT,,,Null,No,ADDACTIONTYPE --> CDORGTP,ADDACTIONTYPE,ORG_TP_CD,CDORGTP,ORG_TP_CD
4,4,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,ADDACTIONTYPE,The ADDACTIONTYPE table identifies an action t...,PERSON_ORG_CODE,Indicates the type of CONTACT - person or orga...,CHAR(1),,,Not Null,No,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939,,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,USERTABLE,A type of PERSON that is allowed to use the ap...,USER_ID,The unique user ID to identify the system user.\,VARCHAR(30),,,Not Null,No,None,None,None,None,None
1940,,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,USERTABLE,A type of PERSON that is allowed to use the ap...,PASSWORD,The password used by the user to authenticate ...,VARCHAR(20,,,Null,No,None,None,None,None,None
1941,,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,USERTABLE,A type of PERSON that is allowed to use the ap...,EXPIRY_DT,The date that the user is no longer valid.,TIMESTAMP,,,Null,No,None,None,None,None,None
1942,,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,USERTABLE,A type of PERSON that is allowed to use the ap...,LAST_UPDATE_DT,"When a record is added or updated, this field ...",TIMESTAMP,,,Not Null,No,None,None,None,None,None


In [25]:
1944*18

34992

In [8]:
def meta_data_package_template(field_d):
    dc_terms_base = "http://purl.org/dc/terms/"
    rdf_form = {}
    for k,v in field_d.items():
        rdf_form[URIRef(dc_terms_base + k)] = Literal(v)
    return rdf_form



d_p = meta_data_package_template( {"created" : "2023-04-01", 
                                   "creator" : "tomk", 
                                   "description" : "test data package", 
                                   "title" : "some title", 
                                   "modified" : "2023-04-05"})



In [9]:
rows = [dict({rk:html.escape(str(rv)) for rk, rv in r.items()}) for i,r in model_data.iterrows()]

In [10]:
import loader

In [11]:
rows[0]

{'Sequence': '0.0',
 'ModelDomain': 'IBM MDM',
 'Model': 'IBM MDM Physical Datamodel v11.3_extract_1',
 'ModelType': 'Physical',
 'Class': 'ADDACTIONTYPE',
 'ClassDescription': 'The ADDACTIONTYPE table identifies an action taken as a result of suspect duplicate identification. This table is used by the following domain. v Party Domain ',
 'Attribute': 'ADD_ACTION_ID',
 'AttributeDescription': 'A unique, system-generated key that identifies an add action in the system.',
 'DataType': 'BIGINT',
 'Context': '',
 'SubContext': '',
 'Nulls': 'Not Null',
 'Is PK': 'Yes',
 'Relationship': 'None',
 'FromClass': 'None',
 'FromAttribute': 'None',
 'ToClass': 'None',
 'ToAttribute': 'None'}

In [12]:
if len(sg)==0:
    sg.parse("DMEAR_ser.rdf")

In [13]:
# Here master_g is a master graph sourced from Fuseki
mq,me_m = loader.get_triples(sg, "EntityAttributeRecord", rows, master_g, [])

0:00:10.384271 for 1944


In [16]:
def generate_discourse(d_name, master_triples, payload, master_uri=URIRef("http://master"), discourse_uri=URIRef("http://discourse")):
    posits = set()
    declarations = set()
    for triple in master_triples:
        t_pos = discourse.Posit(triple)
        posits.add(t_pos)
        t_dec = discourse.Declaration(t_pos.uri, asserts=True)
        declarations.add(t_dec)
        
    t_disc = discourse.Discourse(d_name, payload)
    for t_dec in declarations:
        t_disc.add_member_uri(t_dec.uri)

    master_triples = set([(s,p,o) for s,p,o in master_triples])
    posit_triples = set(chain(*[p.to_triples() for p in posits]))
    declaration_triples = set(chain(*[p.to_triples() for p in declarations]))
    discourse_triples = set(t_disc.to_triples())

    # Need to return separate sets of triples, or attach quad graph names to these ones so they can be 
    # separated out later. 
    return master_triples, posit_triples, declaration_triples, discourse_triples
    
    


In [17]:
m,p,de,di = generate_discourse("test_123", mq, d_p)

In [18]:
len(m), len(p), len(de), len(di)

(13775, 96425, 55100, 13783)

In [30]:
Counter([t[1] for t in di])

Counter({rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#DiscourseContains'): 13775,
         rdflib.term.URIRef('http://purl.org/dc/terms/created'): 1,
         rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'): 1,
         rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'): 1,
         rdflib.term.URIRef('http://purl.org/dc/terms/title'): 1,
         rdflib.term.URIRef('http://purl.org/dc/terms/creator'): 1,
         rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#GeneratedOn'): 1,
         rdflib.term.URIRef('http://purl.org/dc/terms/description'): 1,
         rdflib.term.URIRef('http://purl.org/dc/terms/modified'): 1})

In [24]:
list(m)[0]

(rdflib.term.URIRef('http://www.semanticweb.org/tomk/ontologies/2022/11/6fb7e149f4c04746b2eb4488fff193d5'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
 rdflib.term.Literal('DOMAIN_TP_CD'))

In [28]:
55/13

4.230769230769231

In [ ]:


posits=set()
declarations=set()

for t in mq: # Step over a set of triples

    pp = discourse.Posit(t[0:3]) # For each triple, generate a posit (danger of duplication!)
    posits.add(pp)
    dd = discourse.Declaration(pp.uri, asserts=True) # For each posit, generate a declaration
    declarations.add(dd)

    # Once complete, generate a "Discourse" to act as a container for the set
dc = discourse.Discourse("test_123", payload=d_p) 

for dd in declarations:
    dc.add_member_uri(dd.uri)
    
dc_triples = set(dc.to_triples())


In [ ]:
def c_getter(content):
    for c in content:
        yield c

In [ ]:
len(mq)

In [ ]:
mq[33]

In [ ]:
# Bring all triples together into a single set
# The contents of this set will be spread across different named graphs in the database.
# mq_triples --> master
# posit_triples --> 
mq_triples = set([(s,p,o) for s,p,o,g in mq])
posit_triples = set(chain(*[p.to_triples() for p in posits]))
declaration_triples = set(chain(*[p.to_triples() for p in declarations]))
discourse_triples = set(dc.to_triples())

all_triples = set(mq_triples).union(posit_triples.union(declaration_triples).union(discourse_triples))
print(len(all_triples))


In [ ]:

g_g = c_getter(all_triples)

pred_list = []
subj_list = []
obj_list = []
for s,p,o in g_g:
    pred_list.append(p)
    subj_list.append(s)
    obj_list.append(o)
p_counter = Counter(pred_list)
subj_counter = Counter(subj_list)
obj_counter = Counter(obj_list )


In [ ]:
p_counter

In [ ]:
assert False

In [ ]:
import hashlib

In [ ]:
def uuid_format(hx):
    return hx[0:12]+"-"+hx[12:16]+"-"+hx[16:20]+"-"+hx[20:]

In [ ]:
uuid_format(hashlib.md5(b"test").hexdigest()), str(uuid.uuid4())


In [ ]:
uuid_format(hashlib.md5(pp.to_triples()[-1][2].n3().encode("utf-8")).hexdigest())

In [ ]:
pp.to_triples()

In [ ]:

lq = loader.master_on_predicate_g(mq, tq)
master_g.addN(lq)

In [ ]:
def quads_to_triples(qs):
    return list(((t[0], t[1], t[2]) for t in qs))

def rebase_quads(qs, base):
    return list(((t[0], t[1], t[2], base) for t in qs))

In [ ]:

ds.addN(rebase_quads(tq, URIRef("http://test")))
test_g = ds.graph(URIRef("http://test"))
len(test_g)

In [ ]:
master_g.commit()

In [ ]:
mq

In [ ]:
ds.add((URIRef("test"), URIRef("test"), URIRef("test"), URIRef("http://test")))

In [ ]:
len(test_g)